<a href="https://colab.research.google.com/github/reutDayan1/reutDayan1/blob/main/reut_dayan_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
%load_ext tensorboard
import datetime, os
import tensorflow as tf
import tensorflow_datasets as tfds

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [32]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
     split=['train', 'test'],
     shuffle_files=True,
     as_supervised=True,
     with_info=True,
)
num_of_classes = ds_info.features['label'].num_classes
image_shape = ds_info.features['image'].shape
train_size = ds_info.splits['train'].num_examples
test_size = ds_info.splits['test'].num_examples

print(num_of_classes)
print(image_shape)
print(train_size)
print(test_size)

10
(28, 28, 1)
60000
10000


In [33]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)


ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [34]:
layers = [
    tf.keras.layers.Flatten(input_shape=image_shape),


    tf.keras.layers.Dense(10),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('sigmoid'),


    tf.keras.layers.Dense(92),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('sigmoid'),


    tf.keras.layers.Dense(87),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('sigmoid'),


    tf.keras.layers.Dense(18),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('sigmoid'),


    tf.keras.layers.Dense(num_of_classes),
    tf.keras.layers.Softmax()
]










In [35]:
model = tf.keras.models.Sequential(layers)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
               metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_6 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 10)                  │           7,850 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_20               │ (None, 10)                  │              40 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_24 (Activation)           │ (None, 10)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 92)                  │           1,012 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_21               │ (None, 92)                  │             368 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_25 (Activation)           │ (None, 92)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_32 (Dense)                     │ (None, 87)                  │           8,091 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_22               │ (None, 87)                  │             348 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_26 (Activation)           │ (None, 87)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 18)                  │           1,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_23               │ (None, 18)                  │              72 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_27 (Activation)           │ (None, 18)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_34 (Dense)                     │ (None, 10)                  │             190 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax_6 (Softmax)                  │ (None, 10)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,555 (76.39 KB)

 Trainable params: 19,141 (74.77 KB)

 Non-trainable params: 414 (1.62 KB)

In [36]:

history = model.fit(
    ds_train,
    epochs=50,
    validation_data=ds_test,
)

best_epoch_value = max(history.history['val_sparse_categorical_accuracy'])
best_epoch_index = history.history['val_sparse_categorical_accuracy'].index(best_epoch_value) + 1
print(f"Best performance at epoch {best_epoch_index} with validation sparse categorical accuracy of {best_epoch_value:.4f}")


Epoch 1/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 74s 3ms/step - loss: 1.2231 - sparse_categorical_accuracy: 0.7602 - val_loss: 0.3324 - val_sparse_categorical_accuracy: 0.9099
Epoch 2/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 0.3238 - sparse_categorical_accuracy: 0.9095 - val_loss: 0.2488 - val_sparse_categorical_accuracy: 0.9242
Epoch 3/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.2580 - sparse_categorical_accuracy: 0.9226 - val_loss: 0.1962 - val_sparse_categorical_accuracy: 0.9418
Epoch 4/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.2324 - sparse_categorical_accuracy: 0.9298 - val_loss: 0.1802 - val_sparse_categorical_accuracy: 0.9433
Epoch 5/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.2137 - sparse_categorical_accuracy: 0.9340 - val_loss: 0.1836 - val_sparse_categorical_accuracy: 0.9422
Epoch 6/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.1990 - sparse_categorical_accuracy: 0.9384 - val_loss: 0.1699 - val_sparse_categorical_accura